In [50]:
'''
    To be used in environment.py
'''


import numpy as np

def generate_box_rotations(boxes, rotation_indices = None):                                     # Based on this coordinate system:
    rotations = np.array([                                                                      # z
        [0, 1, 2],  # 0: (x, y, z) --> Original State                                           # ^
        [1, 0, 2],  # 1: (y, x, z) --> Box rotated 90° around the height axis (z)               # |
        [2, 1, 0],  # 2: (z, y, x) --> Box tipped forward/backward                              # |_____> y
        [1, 2, 0],  # 3: (y, z, x) --> Box tipped forward/backward and then rotated 90°         #  \
        [0, 2, 1],  # 4: (x, z, y) --> Box tipped to the left or right                          #   \
        [2, 0, 1]   # 5: (z, x, y) --> Box tipped to the left or right and then rotated 90°     #    _|
    ])                                                                                          #      x
                                                                                                #
    if rotation_indices is not None:                                                            #       ^
        rotations = rotations[rotation_indices]                                                 #       |
                                                                                                #     Viewer
    return boxes[:, rotations]

boxes = np.array([[6, 7, 8]])
rotations_1 = None
rotations_2 = [1]
# rotations_3 = [[1], [2, 3]] # <-- Does not work, but it is not needed in environment.py because either no rotation_constraints are used (get_packing_mask()) or only one rotation is passed as in (step()).


test_1 = generate_box_rotations(boxes, rotations_1)
test_2 = generate_box_rotations(boxes, rotations_2)
# test_3 = generate_box_rotations(boxes, rotations_3)

print(test_1)
print()
print(test_2)
print()
# print(test_3)

[[[6 7 8]
  [7 6 8]
  [8 7 6]
  [7 8 6]
  [6 8 7]
  [8 6 7]]]

[[[7 6 8]]]



In [ ]:
'''
    To be used in network.py
'''


import numpy as np
import torch

def generate_box_rotations_torch(boxes, rotation_constraints = None):
    
    if isinstance(boxes, np.ndarray):
        boxes = torch.tensor(boxes)
    
    device = boxes.device
    batch_size, box_num, _ = boxes.shape

    base_rotations = torch.tensor([
        [0, 1, 2],
        [1, 0, 2],
        [2, 1, 0],
        [1, 2, 0],
        [0, 2, 1],
        [2, 0, 1]
    ], device = device)

    # Case 1: All rotations are allowed
    if rotation_constraints is None:
        allowed_rot = base_rotations
        num_rot = 6

    elif isinstance(rotation_constraints[0], list):
        # Case 2: Same rotation for all boxes
        if len(rotation_constraints) == 1:
            r_tensor = base_rotations[rotation_constraints[0]]
            allowed_rot = r_tensor.unsqueeze(0).expand(box_num, -1, -1)  # für alle Boxen
            num_rot = allowed_rot.shape[1]
        else:
            # Case 3:Individual rotation constraints per box
            max_rot = max(len(r) for r in rotation_constraints)
            allowed_rot_list = []
            for r in rotation_constraints:
                r_tensor = base_rotations[r]
                if len(r) < max_rot:
                    pad = torch.zeros((max_rot - len(r), 3), device=device, dtype=torch.long)
                    r_tensor = torch.cat([r_tensor, pad], dim=0)
                allowed_rot_list.append(r_tensor)
            allowed_rot = torch.stack(allowed_rot_list, dim=0)
            num_rot = allowed_rot.shape[1]

    else:
        raise ValueError(
                f"rotation_constraints must be None, list[int] --> [x, y, z] <--, or list[list[int]] --> [[v, w], [x, y, z]] <--. "
                f"Got {rotation_constraints}"
            )

    # Expand for Batch
    boxes_expand = boxes.unsqueeze(2).expand(-1, -1, num_rot, -1)
    if allowed_rot.dim() == 2:
        allowed_rot_expand = allowed_rot.unsqueeze(0).unsqueeze(0).expand(batch_size, box_num, -1, -1)
    else:
        allowed_rot_expand = allowed_rot.unsqueeze(0).expand(batch_size, -1, -1, -1)

    return torch.gather(boxes_expand, dim = 3, index = allowed_rot_expand)           # Gather aloth the last dimension (x, y, z)




boxes = np.array([[[0, 1, 2],
                   [6, 7, 8]]])


# Test cases
rotations_1 = None
rotations_2 = [[1]]
rotations_3 = [[1], [2, 3]]

test_1 = generate_box_rotations_torch(boxes, rotations_1)
test_2 = generate_box_rotations_torch(boxes, rotations_2)
test_3 = generate_box_rotations_torch(boxes, rotations_3)

print(test_1)
print()
print(test_2)
print()
print(test_3)


tensor([[[[0, 1, 2],
          [1, 0, 2],
          [2, 1, 0],
          [1, 2, 0],
          [0, 2, 1],
          [2, 0, 1]],

         [[6, 7, 8],
          [7, 6, 8],
          [8, 7, 6],
          [7, 8, 6],
          [6, 8, 7],
          [8, 6, 7]]]])

tensor([[[[1, 0, 2]],

         [[7, 6, 8]]]])

tensor([[[[1, 0, 2],
          [0, 0, 0]],

         [[8, 7, 6],
          [7, 8, 6]]]])


In [63]:
# Der gleiche Quark, aber mit einer Maske für illegale Rotationen:


def generate_box_rotations_torch(boxes, rotation_constraints=None):
    if isinstance(boxes, np.ndarray):
        boxes = torch.tensor(boxes)
    device = boxes.device
    batch_size, box_num, _ = boxes.shape

    base_rotations = torch.tensor([
        [0,1,2],[1,0,2],[2,1,0],
        [1,2,0],[0,2,1],[2,0,1]
    ], device=device)

    if rotation_constraints is None:
        allowed_rot = base_rotations
        num_rot = 6
        rotation_mask = torch.zeros(batch_size, box_num, num_rot, dtype=torch.bool, device=device)
    elif isinstance(rotation_constraints[0], int):
        allowed_rot = base_rotations[rotation_constraints]
        num_rot = allowed_rot.shape[0]
        rotation_mask = torch.zeros(batch_size, box_num, num_rot, dtype=torch.bool, device=device)
    elif isinstance(rotation_constraints[0], list):
        max_rot = max(len(r) for r in rotation_constraints)
        allowed_rot_list = []
        rotation_mask_list = []
        for r in rotation_constraints:
            r_tensor = base_rotations[r]
            allowed_rot_list.append(r_tensor)
            mask = torch.zeros(max_rot, dtype=torch.bool, device=device)
            mask[len(r):] = True  # ungültige Slots
            rotation_mask_list.append(mask)
        allowed_rot = torch.stack(allowed_rot_list, dim=0)
        rotation_mask = torch.stack(rotation_mask_list, dim=0).unsqueeze(1).expand(batch_size, box_num, -1)
        num_rot = allowed_rot.shape[1]
    else:
        raise ValueError("rotation_constraints invalid")

    # Expand für Batch
    boxes_expand = boxes.unsqueeze(2).expand(-1, -1, num_rot, -1)
    if allowed_rot.dim() == 2:
        allowed_rot_expand = allowed_rot.unsqueeze(0).unsqueeze(0).expand(batch_size, box_num, -1, -1)
    else:
        allowed_rot_expand = allowed_rot.unsqueeze(0).expand(batch_size, -1, -1, -1)

    box_rotations = torch.gather(boxes_expand, dim=3, index=allowed_rot_expand)

    return box_rotations, rotation_mask




boxes = np.array([[[0, 1, 2],
                   [6, 7, 8]]])


# Test cases
rotations_1 = None
rotations_2 = [[1]]
rotations_3 = [[1], [2, 3]]

test_1 = generate_box_rotations_torch(boxes, rotations_1)
test_2 = generate_box_rotations_torch(boxes, rotations_2)
test_3 = generate_box_rotations_torch(boxes, rotations_3)

print(test_1)
print()
print(test_2)
print()


RuntimeError: stack expects each tensor to be equal size, but got [1, 3] at entry 0 and [2, 3] at entry 1